### SocialAL Models
# Data simulation and parameter recovery - multiple subjects
KLS 8.30.19; update 7.7.22  
Project info: https://osf.io/b48n2/

Model modified from :
Fareri, D. S., Chang, L. J., & Delgado, M. R. (2012). Effects of direct social experience on trust decisions and neural reward circuitry. Frontiers in Neuroscience, 6, 1–17. https://doi.org/10.3389/fnins.2012.00148

### Python version

In [1]:
import sys
print(sys.version)  

3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


### Load modules

In [2]:
import numpy as np
import random
import math
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import beta
from scipy.stats import gamma
import matplotlib.pyplot as plt
import itertools

## Pull in functions

In [3]:
run common_functions.ipynb

In [4]:
run double_alpha_functions.ipynb

## Functions to simulate data

In [5]:
def select_response(prob):     
    n = random.uniform(0,1)
    if n > prob:
        response = 0
    else:
        response = 1
    return response

### Set constants

In [6]:
recip_rates = {0: 0.93, 1:0.6, 2:0.07}

### New function to simulate data for one sub

In [7]:
def sim_data(tn, params):
    # tn: number of trials desired
    # params: ground truth of parameters
    a_gain = params[0]
    a_loss = params[1]
    beta = params[2]
    
    # initialize variables
    Probs = [0.5, 0.5, 0.5]
    EVs = [[9,9,9,9],[9,9,9,9],[9,9,9,9]]
    
    # generate trial sequence
    trial_sequence = np.repeat([0,1,2], tn)
    random.shuffle(trial_sequence) #print(trial_sequence)
    
    trial = []
    choices = []
    responses = []
    
    for x in range(0,len(trial_sequence)):
        t = trial_sequence[x] 
        
        # Trial
        trial.append(x+1)
        
        # Make a choice
        choice = action_selection(get_action_selection_probs(beta, EVs[t]))
        choices.append(choice) 
    
        # Get a response
        recip_rate = recip_rates.get(t) 
        
        response = select_response(recip_rate) 
        responses.append(response)
    
        # after choice, update probability
        if choice != 1:
            Probs[t] = update_prob(response, Probs[t], a_gain, a_loss) 
        # then update value
        EVs[t] = update_value(Probs[t]) 

    data = {'Trial': trial, 'Stim_Sequence': trial_sequence, 'Choice' : choices, 'Trustee_Response': responses}    
    return data

In [9]:
#sim_data(15,[.2,.3,2])

### Create set of parameters
- distribution of alpha and beta parameters modeled following Cutler et al., 2021

In [17]:
a_gains = beta.rvs(a = 1.1, b = 1.1, size = 150)
a_losses = beta.rvs(a = 1.1, b = 1.1, size = 150)
betas = gamma.rvs(a = 1.2, scale = 5, size = 150)

### Simulate data

In [18]:
data = pd.DataFrame(columns= ['Trial', 'Stim_Sequence', 'Choice', 'Trustee_Response', 'Subject', 'Alpha_gain', 'Alpha_loss','Beta'])
for p in range(len(betas)):
    dt = sim_data(15, [a_gains[p], a_losses[p], betas[p]])
    dt['Subject'] = [p + 1] * 45
    dt['Alpha_gain'] = [a_gains[p]] * 45
    dt['Alpha_loss'] = [a_losses[p]] * 45
    dt['Beta'] = [betas[p]] * 45
    dt = pd.DataFrame(dt)
    data = pd.concat([data, dt])
data

data.to_csv(path_or_buf = '../../output/simulation/sim_2alpha_model_data.csv', index = False)

### New function to fit model to multiple subjects

In [19]:
def model_fit_mult_subject(data):
    pdt = pd.DataFrame(columns = ['Subject', 'a_gain', 'a_loss', 'beta'])
    a = pd.unique(data['Subject'])
    print('Number of Subs: ', len(a))
    for sub in range(1,len(a)+1):
        print('Subject: ', sub)
        df = data[data['Subject']==sub]
        dt = df.to_dict()
        #print(dt)
        params = model_fit(dt)
        line = {'Subject': sub, 'a_gain': params[0], 'a_loss': params[1], 'beta':params[2]}
        pdt = pdt.append(line, ignore_index=True)   
    return(pdt)


In [20]:
precover = model_fit_mult_subject(data)

Number of Subs:  150
Subject:  1
Subject:  2
Subject:  3
Subject:  4
Subject:  5
Subject:  6
Subject:  7
Subject:  8
Subject:  9
Subject:  10
Subject:  11
Subject:  12
Subject:  13
Subject:  14
Subject:  15
Subject:  16
Subject:  17
Subject:  18
Subject:  19
Subject:  20
Subject:  21
Subject:  22
Subject:  23
Subject:  24
Subject:  25
Subject:  26
Subject:  27
Subject:  28
Subject:  29
Subject:  30
Subject:  31
Subject:  32
Subject:  33
Subject:  34
Subject:  35
Subject:  36
Subject:  37
Subject:  38
Subject:  39
Subject:  40
Subject:  41
Subject:  42
Subject:  43
Subject:  44
Subject:  45
Subject:  46
Subject:  47
Subject:  48
Subject:  49
Subject:  50
Subject:  51
Subject:  52
Subject:  53
Subject:  54
Subject:  55
Subject:  56
Subject:  57
Subject:  58
Subject:  59
Subject:  60
Subject:  61
Subject:  62
Subject:  63
Subject:  64
Subject:  65
Subject:  66
Subject:  67
Subject:  68
Subject:  69
Subject:  70
Subject:  71
Subject:  72
Subject:  73
Subject:  74
Subject:  75
Subject:  76


In [21]:
print(precover)

     Subject    a_gain    a_loss       beta
0        1.0  0.950924  1.000000  12.947741
1        2.0  0.014621  1.000000  20.000000
2        3.0  0.073252  1.000000   2.234138
3        4.0  1.000000  1.000000  14.212956
4        5.0  0.440958  1.000000  17.623392
5        6.0  1.000000  1.000000  14.922716
6        7.0  1.000000  0.000000   7.877827
7        8.0  1.000000  1.000000  16.413788
8        9.0  0.690939  1.000000   7.665917
9       10.0  1.000000  1.000000  14.513516
10      11.0  1.000000  0.181691   7.242920
11      12.0  1.000000  0.000000   6.384589
12      13.0  1.000000  0.000000   4.440369
13      14.0  1.000000  1.000000  17.263532
14      15.0  1.000000  0.382749  20.000000
15      16.0  0.978443  1.000000  13.598957
16      17.0  0.520987  1.000000  11.664573
17      18.0  1.000000  1.000000  12.867506
18      19.0  0.049845  1.000000   5.173105
19      20.0  1.000000  1.000000  16.718182
20      21.0  1.000000  0.000000  17.351074
21      22.0  1.000000  0.000000

In [22]:
precover.to_csv(path_or_buf = '../../output/simulation/sim_2alpha_model_fit.csv', index = False)